In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files
files.upload()  # выберите kaggle.json

In [ ]:
  !mkdir ~/.kaggle
  !mv kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json

  !kaggle datasets download -d olistbr/brazilian-ecommerce
  !unzip brazilian-ecommerce.zip


In [ ]:
!ls /content

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import sqlite3


df_olist_customers = pd.read_csv('/content/olist_customers_dataset.csv')
df_olist_sellers = pd.read_csv('/content/olist_sellers_dataset.csv')
df_olist_order_reviews= pd.read_csv('/content/olist_order_reviews_dataset.csv')
df_olist_order_items= pd.read_csv('/content/olist_order_items_dataset.csv')
df_olist_products= pd.read_csv('/content/olist_products_dataset.csv')
df_olist_geolocation= pd.read_csv('/content/olist_geolocation_dataset.csv')
df_product_category_name_translation= pd.read_csv('/content/product_category_name_translation.csv')
df_olist_orders = pd.read_csv('/content/olist_orders_dataset.csv')
df_olist_order_payments= pd.read_csv('/content/olist_order_payments_dataset.csv')

df_olist_customers.head()

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export the dataframe as a table 'playstore' to the sqlite engine
df_olist_customers.to_sql("olist_customers", con =engine)
df_olist_sellers.to_sql("olist_sellers", con =engine)
df_olist_order_reviews.to_sql("olist_order_reviews", con =engine)
df_olist_order_items.to_sql("olist_order_items", con =engine)
df_olist_products.to_sql("olist_products_dataset", con =engine)
df_olist_geolocation.to_sql("olist_geolocation", con =engine)
df_product_category_name_translation.to_sql("product_category_name_translation", con =engine)
df_olist_orders.to_sql("olist_orders", con =engine)
df_olist_order_payments.to_sql("olist_order_payments", con =engine)
df_olist_order_payments.head()

In [ ]:
# 2. Create SQLite Database and Export DataFrames.
from sqlalchemy import create_engine

# Creating an SQLite engine (in-memory database)
engine = create_engine('sqlite://', echo=False)

# Export all DataFrames as SQL tables
df_olist_customers.to_sql("olist_customers", con=engine, index=False)
df_olist_sellers.to_sql("olist_sellers", con=engine, index=False)
df_olist_order_reviews.to_sql("olist_order_reviews", con=engine, index=False)
df_olist_order_items.to_sql("olist_order_items", con=engine, index=False)
df_olist_products.to_sql("olist_products", con=engine, index=False)
df_olist_geolocation.to_sql("olist_geolocation", con=engine, index=False)
df_product_category_name_translation.to_sql("product_category_name_translation", con=engine, index=False)
df_olist_orders.to_sql("olist_orders", con=engine, index=False)
df_olist_order_payments.to_sql("olist_order_payments", con=engine, index=False)


In [ ]:
# SQL query: select all columns from the olist_customers table, limiting it to 5 rows
sql = '''
SELECT *
FROM olist_customers
LIMIT 5
'''

# Executing an SQL query using the engine
df_sql = pd.read_sql_query(sql, con=engine)

# Viewing the result
df_sql.head()


In [ ]:
# 3. Query 1: Count and Percentage of Orders Purchased in Jan 2018 with 5 Review Score
sql_q1 = '''
SELECT
    COUNT(o.order_id) AS num_orders,
    ROUND(CAST(COUNT(o.order_id) AS FLOAT) /
    (SELECT COUNT(*)
    FROM olist_orders
    WHERE strftime('%Y-%m', order_purchase_timestamp) = '2018-01') * 100, 2)
    AS percentage
FROM olist_orders o
JOIN olist_order_reviews r ON o.order_id = r.order_id

WHERE
strftime('%Y-%m', o.order_purchase_timestamp) = '2018-01'
AND r.review_score = 5
'''
df_q1 = pd.read_sql_query(sql_q1, con=engine)
df_q1


In [ ]:
# 4. Query 2: Customer Purchase Trend Year-on-Year
sql_q2 = '''
SELECT
    strftime('%Y', order_purchase_timestamp) AS year,
    COUNT(order_id) AS num_orders,
    COUNT(DISTINCT customer_id) AS num_customers
FROM olist_orders
GROUP BY year
ORDER BY year
'''
df_q2 = pd.read_sql_query(sql_q2, con=engine)
df_q2


In [ ]:
# 5. Query 3: Average Order Values of Customers
sql_q3 = '''
SELECT
    o.customer_id,
    ROUND(AVG(p.payment_value), 2) AS avg_order_value
FROM olist_orders o
JOIN olist_order_payments p ON o.order_id = p.order_id
GROUP BY o.customer_id

ORDER BY avg_order_value DESC
LIMIT 10
'''
df_q3 = pd.read_sql_query(sql_q3, con=engine)
df_q3


In [ ]:
# 6. Query 4: Top 5 Cities with Highest Revenue from 2016 to 2018
sql_q4 = '''
SELECT
    c.customer_city,
    ROUND(SUM(p.payment_value), 2) AS total_revenue
FROM olist_orders o
JOIN olist_order_payments p ON o.order_id = p.order_id
JOIN olist_customers c ON o.customer_id = c.customer_id

WHERE strftime('%Y', o.order_purchase_timestamp) BETWEEN '2016' AND '2018'
GROUP BY c.customer_city

ORDER BY total_revenue DESC
LIMIT 5
'''
df_q4 = pd.read_sql_query(sql_q4, con=engine)
df_q4


In [ ]:
# 7. Query 5: State Wise Revenue Table Between 2016 to 2018
sql_q5 = '''
SELECT
    c.customer_state,
    ROUND(SUM(p.payment_value), 2) AS total_revenue
FROM olist_orders o
JOIN olist_order_payments p ON o.order_id = p.order_id
JOIN olist_customers c ON o.customer_id = c.customer_id

WHERE strftime('%Y', o.order_purchase_timestamp) BETWEEN '2016' AND '2018'
GROUP BY c.customer_state

ORDER BY total_revenue DESC
LIMIT 10
'''
df_q5 = pd.read_sql_query(sql_q5, con=engine)
df_q5


In [ ]:
# 8. Query 6: Top Successful Sellers
sql_q6 = '''
SELECT
    s.seller_id,
    COUNT(oi.order_id) AS total_goods_sold,
    ROUND(SUM(oi.price), 2) AS total_revenue,
    COUNT(DISTINCT o.customer_id) AS total_customers,
    SUM(CASE WHEN r.review_score = 5 THEN 1 ELSE 0 END) AS five_star_reviews

FROM olist_sellers s
JOIN olist_order_items oi ON s.seller_id = oi.seller_id
JOIN olist_orders o ON oi.order_id = o.order_id
LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id

GROUP BY s.seller_id

ORDER BY total_goods_sold DESC, total_revenue DESC
LIMIT 10
'''
#LEFT JOIN - Not all orders have a review.
#We want to save all orders, even if there is no review.

df_q6 = pd.read_sql_query(sql_q6, con=engine)
df_q6


In [ ]:
# 9. Query 7: Delivery Success Rate Across States
sql_q7 = '''
SELECT
    c.customer_state,
    ROUND(SUM(CASE WHEN o.order_status='delivered' THEN 1 ELSE 0 END) * 100.0 /
    COUNT(*), 2) AS delivery_success_rate
FROM olist_orders o
JOIN olist_customers c ON o.customer_id = c.customer_id

GROUP BY c.customer_state

ORDER BY delivery_success_rate DESC
LIMIT 10
'''
df_q7 = pd.read_sql_query(sql_q7, con=engine)
df_q7


In [ ]:
# 10. Query 8: Preferred Form of Payment for Different Categories
sql_q8 = '''
SELECT
    pc.product_category_name,
    p.payment_type,
    COUNT(*) AS count_payment
FROM olist_order_items oi
JOIN olist_products pr ON oi.product_id = pr.product_id
JOIN product_category_name_translation pc ON pr.product_category_name = pc.product_category_name
JOIN olist_order_payments p ON oi.order_id = p.order_id

GROUP BY pc.product_category_name, p.payment_type

ORDER BY pc.product_category_name, count_payment DESC
'''
df_q8 = pd.read_sql_query(sql_q8, con=engine)
df_q8

In [20]:
# 11. Query 9: Distance Between Cities
sql_q9 = '''
WITH city_coords AS (
  SELECT
    geolocation_city,
    AVG(geolocation_lat) AS lat,
    AVG(geolocation_lng) AS lng
  FROM olist_geolocation
  GROUP BY geolocation_city
)
SELECT
  c1.geolocation_city AS city_1,
  c2.geolocation_city AS city_2,
  ROUND(6371 * 2 * ASIN(SQRT(
      POWER(SIN(RADIANS(c2.lat - c1.lat) / 2), 2) +
      COS(RADIANS(c1.lat)) * COS(RADIANS(c2.lat)) *
      POWER(SIN(RADIANS(c2.lng - c1.lng) / 2), 2)
  )), 2) AS distance_km
FROM city_coords c1
JOIN city_coords c2
  ON c1.geolocation_city < c2.geolocation_city
LIMIT 10000;

'''
df_q9 = pd.read_sql_query(sql_q9, con=engine)
df_q9

,city_1,city_2,distance_km
0,* cidade,...arraial do cabo,794.80
1,* cidade,4o. centenario,403.41
2,* cidade,4º centenario,403.58
3,* cidade,abadia de goias,979.10
4,* cidade,abadia dos dourados,813.74
...,...,...,...
9995,...arraial do cabo,cotriguacu,2281.70
9996,...arraial do cabo,cotriguaçu,2280.79
9997,...arraial do cabo,couto de magalhaes,1793.24
9998,...arraial do cabo,couto de magalhaes de minas,564.97
